<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Evaluating Classification Models on Humor Styles Data

---

In this lab you will be practicing evaluating classification models (Logistic Regression in particular) on a "Humor Styles" survey.

This survey is designed to evaluate what "style" of humor subjects have. Your goal will be to classify gender using the responses on the survey.

### Humor styles questions encoding reference

### 32 questions:

Subjects answered **32** different questions outlined below:

    1. I usually don't laugh or joke with other people.
    2. If I feel depressed, I can cheer myself up with humor.
    3. If someone makes a mistake, I will tease them about it.
    4. I let people laugh at me or make fun of me at my expense more than I should.
    5. I don't have to work very hard to make other people laugh. I am a naturally humorous person.
    6. Even when I'm alone, I am often amused by the absurdities of life.
    7. People are never offended or hurt by my sense of humor.
    8. I will often get carried away in putting myself down if it makes family or friends laugh.
    9. I rarely make other people laugh by telling funny stories about myself.
    10. If I am feeling upset or unhappy I usually try to think of something funny about the situation to make myself feel better.
    11. When telling jokes or saying funny things, I am usually not concerned about how other people are taking it.
    12. I often try to make people like or accept me more by saying something funny about my own weaknesses, blunders, or faults.
    13. I laugh and joke a lot with my closest friends.
    14. My humorous outlook on life keeps me from getting overly upset or depressed about things.
    15. I do not like it when people use humor as a way of criticizing or putting someone down.
    16. I don't often say funny things to put myself down.
    17. I usually don't like to tell jokes or amuse people.
    18. If I'm by myself and I'm feeling unhappy, I make an effort to think of something funny to cheer myself up.
    19. Sometimes I think of something that is so funny that I can't stop myself from saying it, even if it is not appropriate for the situation.
    20. I often go overboard in putting myself down when I am making jokes or trying to be funny.
    21. I enjoy making people laugh.
    22. If I am feeling sad or upset, I usually lose my sense of humor.
    23. I never participate in laughing at others even if all my friends are doing it.
    24. When I am with friends or family, I often seem to be the one that other people make fun of or joke about.
    25. I donít often joke around with my friends.
    26. It is my experience that thinking about some amusing aspect of a situation is often a very effective way of coping with problems.
    27. If I don't like someone, I often use humor or teasing to put them down.
    28. If I am having problems or feeling unhappy, I often cover it up by joking around, so that even my closest friends don't know how I really feel.
    29. I usually can't think of witty things to say when I'm with other people.
    30. I don't need to be with other people to feel amused. I can usually find things to laugh about even when I'm by myself.
    31. Even if something is really funny to me, I will not laugh or joke about it if someone will be offended.
    32. Letting others laugh at me is my way of keeping my friends and family in good spirits.

---

### Response scale:

For each question, there are 5 possible response codes ("likert scale") that correspond to different answers. There is also a code that indicates there is no response for that subject.

    1 == "Never or very rarely true"
    2 == "Rarely true"
    3 == "Sometimes true"
    4 == "Often true"
    5 == "Very often or always true
    [-1 == Did not select an answer]
    
---

### Demographics:

    age: entered as as text then parsed to an interger.
    gender: chosen from drop down list (1=male, 2=female, 3=other, 0=declined)
    accuracy: How accurate they thought their answers were on a scale from 0 to 100, answers were entered as text and parsed to an integer. They were instructed to enter a 0 if they did not want to be included in research.	

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

### 1. Load the data and perform any EDA and cleaning you think is necessary.

It is worth reading over the description of the data columns above for this.

In [2]:
hsq = pd.read_csv('./hsq_data.csv')

In [3]:
# correct spelling
hsq.rename(columns={'agressive':'aggressive'}, inplace=True)

In [4]:
hsq.shape

(1071, 39)

In [5]:
# looks like there are 4 genders but most are just male and female
print hsq.gender.unique()
print hsq.gender.value_counts()

[2 1 3 0]
1    581
2    477
3      8
0      5
Name: gender, dtype: int64


In [6]:
# Set any of the -1 values in the question answers to np.nan
for col in [c for c in hsq.columns if c.startswith('Q')]:
    hsq[col] = hsq[col].map(lambda x: np.nan if x == -1 else x)

In [7]:
# check null values:
hsq.isnull().sum()

Q1                3
Q2                3
Q3                2
Q4                1
Q5                2
Q6                2
Q7                2
Q8                7
Q9                4
Q10               3
Q11               2
Q12               2
Q13               2
Q14               4
Q15               7
Q16               5
Q17              13
Q18               6
Q19               4
Q20               6
Q21               8
Q22               7
Q23               8
Q24               7
Q25              10
Q26               9
Q27               5
Q28               4
Q29               6
Q30               8
Q31               7
Q32               3
affiliative       0
selfenhancing     0
aggressive        0
selfdefeating     0
age               0
gender            0
accuracy          0
dtype: int64

In [8]:
# drop the nulls
hsq.dropna(inplace=True)
print hsq.shape

(993, 39)


In [9]:
hsq.age.unique()

array([   25,    44,    50,    52,    30,    27,    34,    18,    33,
          26,    36,    21,    20,    23,    70,    17,    39,    61,
          29,    16,    69,    22,    38,    24,    14,    40,    62,
          51,    35,    46,    42,    19,    32,    15,    37,    45,
          28,    49,    31,    64,    54,    68,    48,    60,    43,
          41,    53,    58,   242,   151,    55,    67,    56,    59,
          66,    47,  2670,    57, 44849])

In [10]:
# set hsq to be only valid ages:
hsq = hsq[hsq.age <= 100]

In [11]:
# only keep male and female
hsq = hsq[hsq.gender.isin([1,2])]

### 2. Set up a predictor matrix to predict `gender` (only male vs. female)

Choice of predictors is up to you. Justify which variables you include.

In [12]:
# not including the "aggregate" measures (affiliative, selfenhancing, etc.) as they are combinations
# of the original questions.
predictors = [x for x in hsq.columns if 'Q' in x]
predictors = predictors + ['age', 'accuracy']
print predictors

['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'age', 'accuracy']


In [13]:
# set up y variable
y = hsq.gender.map(lambda x: 1 if x == 1 else 0)

In [14]:
X = hsq[predictors]

### 3. StandardScaler to normalise your data and Fit a Logistic Regression model and compare your cross-validated accuracy to the baseline.

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
Xs = ss.fit_transform(X)

In [16]:
lr = LogisticRegression(C=10**6)
cross_val_score(lr, X, y, cv=25)

array([ 0.6       ,  0.6       ,  0.625     ,  0.625     ,  0.675     ,
        0.625     ,  0.675     ,  0.575     ,  0.525     ,  0.625     ,
        0.53846154,  0.66666667,  0.46153846,  0.64102564,  0.64102564,
        0.56410256,  0.71794872,  0.44736842,  0.65789474,  0.65789474,
        0.71052632,  0.63157895,  0.57894737,  0.44736842,  0.63157895])

In [17]:
np.mean(y)

0.5475946775844421

### 4. Create a 50-50 train-test split. Fit the model on training and get the predictions and predicted probabilities on the test data.

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [19]:
# A:
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.5)
lr.fit(X_train, y_train)

LogisticRegression(C=1000000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [20]:
yhat = lr.predict(X_test)
yhat_pp = lr.predict_proba(X_test)

In [21]:
print yhat[:5]
print yhat_pp[:5]

[0 0 0 0 1]
[[ 0.71179952  0.28820048]
 [ 0.66983916  0.33016084]
 [ 0.63031039  0.36968961]
 [ 0.82007952  0.17992048]
 [ 0.19673802  0.80326198]]


### 5. Manually calculate the true positives, false positives, true negatives, and false negatives.

In [22]:
# A:
tp = np.sum((y_test == 1) & (yhat == 1))
fp = np.sum((y_test == 0) & (yhat == 1))
tn = np.sum((y_test == 0) & (yhat == 0))
fn = np.sum((y_test == 1) & (yhat == 0))
print tp, fp, tn, fn

173 93 115 108


### 6. Construct the confusion matrix. 

In [23]:
# A:
confusion_matrix_ = [[tp,fp],[fn,tn]]

In [24]:
conmat = np.array(confusion_matrix(y_test, yhat, labels=[1,0]))

confusion = pd.DataFrame(conmat, index=['is_male', 'is_female'],
                         columns=['predicted_male','predicted_female'])
confusion.T

,is_male,is_female
predicted_male,173,93
predicted_female,108,115


### 7. Print out the false positive count as you change your threshold for predicting label 1.

In [25]:
pp = pd.DataFrame(yhat_pp, columns = ['female', 'male'])
pp.head()

,female,male
0,0.711800,0.288200
1,0.669839,0.330161
2,0.630310,0.369690
3,0.820080,0.179920
4,0.196738,0.803262


In [26]:
for thresh in np.arange(1,10)/10.:
    labeled_male = np.array([1 if x >= thresh else 0 for x in pp.male.values])
    print 'Threshold:', thresh, 'false positives:', np.sum((y_test == 0) & (labeled_male == 1))

Threshold: 0.1 false positives: 207
Threshold: 0.2 false positives: 196
Threshold: 0.3 false positives: 173
Threshold: 0.4 false positives: 130
Threshold: 0.5 false positives: 93
Threshold: 0.6 false positives: 57
Threshold: 0.7 false positives: 32
Threshold: 0.8 false positives: 10
Threshold: 0.9 false positives: 2


### 8. Plot an ROC curve using your predicted probabilities on the test data.

Calculate the area under the curve.

> *Hint: go back to the lecture to find code for plotting the ROC curve.*

In [27]:
from sklearn.metrics import roc_curve, auc

In [28]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

actuals = lr.predict() 
probas = lr.predict_proba(feature_set)
plt.plot(roc_curve(tp), roc_curve(tn))

TypeError: predict() takes exactly 2 arguments (1 given)

### 9. Cross-validate a logistic regression with grid search

Logistic regression can also use the Ridge penalty. Sklearn's `LogisticRegressionCV` class will help you cross-validate an appropriate regularization strength.



In [32]:
estimator.get_params()

NameError: name 'estimator' is not defined

In [46]:
from sklearn import grid_search
from sklearn.linear_model import Ridge

n_samples = len(X_test)

alphas = np.logspace(-10,10,21)
gs = grid_search.GridSearchCV(
    estimator = Ridge(),
    param_grid = {'alpha': alphas},
    scoring = 'mean_squared_error')

gs.fit(lr,y)

TypeError: Expected sequence or array-like, got estimator LogisticRegression(C=1000000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [35]:
ridge.get_params()

NameError: name 'ridge' is not defined

In [ ]:
# A:

**9.B Calculate the predicted labels and predicted probabilities on the test set with the Ridge logisitic regression.**

In [ ]:
# A:

**9.C Construct the confusion matrix for the Ridge LR.**

In [ ]:
# A:

### 10. Plot the ROC curve for the original and Ridge logistic regressions on the same plot.

Which performs better?

In [ ]:
# A:

### 11. Cross-validate a Lasso logistic regression.

**Remember:**
- `penalty` must be set to `'l1'`
- `solver` must be set to `'liblinear'`

> **Note:** The lasso penalty can be considerably slower. You may want to try fewer Cs or use fewer cv folds.

In [ ]:
# A:

### 12. Make the confusion matrix for the Lasso model.

In [ ]:
# A:

### 13. Plot all three logistic regression models on the same ROC plot.

Which is the best? (if any)

In [ ]:
# A:

### 14. Look at the coefficients for the Lasso logistic regression model. Which variables are the most important?

In [ ]:
# A: